<h1> Machine Learning w chmurze - Cloud ML </h1>

W tym notebooku pokażemy jak przenieść prosty model Tensorflow do GCP i uruchomić na nim predykcje

<h2> Predykcje na podstawie tekstu </h2>

<b>Źródło danych</b>: Yelp Restaurant Reviews (https://www.yelp.com/dataset/challenge)

Dataset zawiera między innymi informacje o restauracjach oraz opinie klientów

Zadaniem jest przewidzenie czy restauracje przejdą inspekcje (amerykańskiego) sanepidu na podstawie opinii gości oraz dodatkowych informacji takich jak lokalizacja i rodzaje kuchni serwowanych w restauracji.

<h2> Ustawienie zmiennych środowiskowych, import bibliotek </h2>

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
BUCKET = 'dswbiznesie'
PROJECT = 'dswbiznesie'
REGION = 'europe-west1'

In [3]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [4]:
%datalab project set -p $PROJECT

In [26]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [5]:
import google.datalab.ml as ml
import tensorflow as tf
import apache_beam as beam
import shutil
import datetime
from apache_beam.io.gcp.internal.clients import bigquery
import google.datalab.bigquery as bq
print tf.__version__

1.2.1


/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-11-07 21:55:59.120932. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


<h2> Dane źródłowe </h2>


Dataset pobrany ze strony Yelp zawiera następujące pliki

In [65]:
!gsutil -q ls -l gs://$BUCKET/rawdata/hygiene

 103859986  2017-11-02T23:29:21Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat
    762559  2017-11-02T23:29:22Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat.additional
     90363  2017-11-02T23:29:23Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat.labels
TOTAL: 3 objects, 104712908 bytes (99.86 MiB)


* <b>hygiene.dat</b>: każda linia zawiera połączone opinie klientów danej restauracji
* <b>hygiene.dat.labels</b>: dla pierwszych 546 linii przypisana jest dodatkowe pole w którym 0 oznacza to że restauracja przeszła inspekcje, 1 to że restauracja <b>nie</b> przeszła inspekcji. Reszta wpisów posiada wpis "[None]" co oznacza że należą do zbioru testowego
* <b>hygiene.dat.additional</b>: plik CSV gdzie w pierwszym polu znajduje się lista oferowanych rodzajów kuchnii, w drugim kod pocztowy restauracji który można uznać za przybliżenie lokalizacji restauracji. W trzecim polu znajduje się liczba opinii, w czwartym średnia ocena ( w skali 0-5, gdzie 5 oznacza ocene najlepszą).

<h2> Feature engineering używając Apache Beam i BigQuery</h2>

Dane źródłowe należy przetowrzyć i dostosować do postaci której będzie można łatwo użyć do uczenia i ewaluacji modelu. 
Najwygodniejszym choć nie najtańszym rozwiązaniem jest załadowanie danych do BigQuery.

Odpowiednim narzędziem do tego zadania jest Apache Beam i jego implementacja - Google Dataflow.
Job Dataflow uruchamiany jest w chmurze. Jego przebieg można śledzić w Konsoli GCP (https://console.cloud.google.com/dataflow).
Uruchomienie joba trwa powyżej minuty.

In [76]:
#clear BigQuery table here

In [17]:
def create_record(rest_tuple):
    #print(rest_tuple)
    identity = rest_tuple[0]
    reviews = rest_tuple[1]['reviews_kv'][0]
    inspection_result = int(rest_tuple[1]['labels_kv'][0]) if rest_tuple[1]['labels_kv'][0] != "[None]" else None
    categories_temp = rest_tuple[1]['attributes_kv'][0].split("\"")
    categories = [ x.replace('\'', '').replace('[','').replace(']','') for x in categories_temp[1].split(',')]
    attributes_temp = categories_temp[2].split(",")
    zip_code = attributes_temp[1]
    review_count = int(attributes_temp[2])
    avg_rating = float(attributes_temp[3])
    
    return { 
        'identity': identity, 
        'reviews': reviews,
        'inspection_result': inspection_result,
        'categories': categories,
        'zip_code': zip_code,
        'review_count': review_count,
        'avg_rating': avg_rating
    }

def preprocess(RUNNER,BUCKET,BIGQUERY_TABLE):
    job_name = 'hygiene-ftng' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
    print 'Launching Dataflow job {} ... hang on'.format(job_name)
    OUTPUT_DIR = 'gs://{0}/data/hygiene/'.format(BUCKET)
    options = {
        'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
        'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
        'job_name': 'hygiene-ftng' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
        'project': PROJECT,
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True
    }
    opts = beam.pipeline.PipelineOptions(flags=[], **options)
    p = beam.Pipeline(RUNNER, options=opts)
    
    # Adding table definition
    table_schema = bigquery.TableSchema()
    
    # Fields definition
    identity_schema = bigquery.TableFieldSchema()
    identity_schema.name = 'identity'
    identity_schema.type = 'integer'
    identity_schema.mode = 'required'
    table_schema.fields.append(identity_schema)
    
    
    reviews_schema = bigquery.TableFieldSchema()
    reviews_schema.name = 'reviews'
    reviews_schema.type = 'string'
    reviews_schema.mode = 'required'
    table_schema.fields.append(reviews_schema)

    inspection_result_schema = bigquery.TableFieldSchema()
    inspection_result_schema.name = 'inspection_result'
    inspection_result_schema.type = 'integer'
    inspection_result_schema.mode = 'nullable'
    table_schema.fields.append(inspection_result_schema)
    
    categories_schema = bigquery.TableFieldSchema()
    categories_schema.name = 'categories'
    categories_schema.type = 'string'
    categories_schema.mode = 'repeated'
    table_schema.fields.append(categories_schema)
    
    zip_code_schema = bigquery.TableFieldSchema()
    zip_code_schema.name = 'zip_code'
    zip_code_schema.type = 'string'
    zip_code_schema.mode = 'required'
    table_schema.fields.append(zip_code_schema)
    
    review_count_schema = bigquery.TableFieldSchema()
    review_count_schema.name = 'review_count'
    review_count_schema.type = 'integer'
    review_count_schema.mode = 'required'
    table_schema.fields.append(review_count_schema)
    
    avg_rating_schema = bigquery.TableFieldSchema()
    avg_rating_schema.name = 'avg_rating'
    avg_rating_schema.type = 'float'
    avg_rating_schema.mode = 'required'
    table_schema.fields.append(avg_rating_schema)
    
    #processing logic
    reviews = p | 'readreviews' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat'.format(BUCKET))
    labels = p | 'readlabels' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat.labels'.format(BUCKET))
    attributes = p | 'readattributes' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat.additional'.format(BUCKET))
    
    reviews_kv = reviews | 'mapreviews to kv' >> beam.Map(lambda x: (x.split(",")[0], ",".join(x.split(",")[1:])))
    labels_kv = labels | 'maplabels to kv' >> beam.Map(lambda x: (x.split(",")[0], x.split(",")[1]))
    attributes_kv = attributes | 'mapattributes to kv' >> beam.Map(lambda x: (x.split(",")[0], x))
    
    restaurants = (
        {'reviews_kv': reviews_kv, 'labels_kv': labels_kv, 'attributes_kv': attributes_kv}
        | 'CoGroupByRestaurantKey' >> beam.CoGroupByKey())
    
    records = restaurants | 'CreateRecords' >> beam.Map(create_record)
    
    records | 'write' >> beam.io.Write(
        beam.io.BigQuerySink(
            BIGQUERY_TABLE,
            schema=table_schema,
            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
            write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE))
    p.run().wait_until_finish()
    print("Done!")

<b> Uruchomienie etl </b>

In [18]:
bigquery_dataset = PROJECT+":"+PROJECT+".hygiene"
preprocess('DirectRunner',BUCKET, bigquery_dataset)

Launching Dataflow job hygiene-ftng-171107-221357 ... hang on
Done!


<h2> Przygotowanie danych do trenowania modelu </h2>

<b> Pobranie danych do DataFrame (pandas) </b>

In [19]:
query="""
SELECT
  identity,
  reviews,
  inspection_result,
  categories,
  zip_code,
  review_count,
  avg_rating
FROM
  `dswbiznesie.hygiene`
WHERE
  inspection_result IS NOT NULL
"""

In [20]:
df = bq.Query(query).execute().result().to_dataframe()

<b> Podzial danych na zestaw treningowy i ewaluacyjny </b>

In [21]:
traindf = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(reviews)),4) > 0").execute().result().to_dataframe()
evaldf  = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(reviews)),4) = 0").execute().result().to_dataframe()
traindf.head()

,identity,reviews,inspection_result,categories,zip_code,review_count,avg_rating
0,357,This restaurant is such a breath of fresh air...,0,"[Seafood, French, Restaurants]",98101,15,4.466667
1,336,I'm a simple girl when it comes to my cocktai...,0,"[American (New), Restaurants]",98101,15,3.866667
2,187,I've only ever had take-out here. Not that it...,0,"[Indian, Himalayan/Nepalese, Restaurants]",98101,5,3.833333
3,433,I walk into most establishments with 5 stars....,0,"[American (Traditional), Breakfast & Brunch, ...",98101,6,2.833333
4,332,I said it before and I'll say it again: Il Fo...,0,"[Delis, Italian, Restaurants]",98101,25,3.576923


In [22]:
traindf['inspection_result'].value_counts()

0    205
1    203
Name: inspection_result, dtype: int64

In [23]:
evaldf['inspection_result'].value_counts()

1    70
0    68
Name: inspection_result, dtype: int64

<h2> Model Tensorflow </h2>

Właściwy model tensorflow znajduje się w pliku <b>model.py</b> a definicja joba Cloud ML w pliku <b>task.py</b>
Kod poniżej ma za zadanie zilutrować działanie kodu tensorflow.

In [24]:
import tensorflow as tf
from tensorflow.contrib import lookup
from tensorflow.python.platform import gfile

print tf.__version__
MAX_DOCUMENT_LENGTH = 100000  
PADWORD = 'ZYXW'

# vocabulary
lines = ['This might be the best "taco" truck on the planet. Hidden between a smoke shop and The Home Depot, this semi permanent, stylish and clean cafe on wheels.', 
         'Im always looking for a good place to sing karaoke. This is one of them. Drinks are cheap. Food is delicious. And its laid back and fun. I shall return!', 
         'Crazy Man just Crazy there are like 3 different places called Saigon Deli at this intersection but this one is on Jackson just west of twelfthand with $2 pork Banh Mi you cant go wrong, I am going to Indiana for work so I went in and got 6 Pork', 
         'Ive eaten here a few times in the past and thought it was decent. I went last night, however, and our meal was really subpar so the place seems to have gone down hill.We had chow fun noodles and the hollow vegetables with chili sauce']

# create vocabulary
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
vocab_processor.fit(lines)
with gfile.Open('vocab.tsv', 'wb') as f:
    f.write("{}\n".format(PADWORD))
    for word, index in vocab_processor.vocabulary_._mapping.iteritems():
      f.write("{}\n".format(word))
N_WORDS = len(vocab_processor.vocabulary_)
print '{} words into vocab.tsv'.format(N_WORDS)

# can use the vocabulary to convert words to numbers
table = lookup.index_table_from_file(
  vocabulary_file='vocab.tsv', num_oov_buckets=1, vocab_size=None, default_value=-1)
numbers = table.lookup(tf.constant(lines[0].split()))
with tf.Session() as sess:
  tf.tables_initializer().run()
  print "{} --> {}".format(lines[0], numbers.eval())   

1.2.1
118 words into vocab.tsv
This might be the best "taco" truck on the planet. Hidden between a smoke shop and The Home Depot, this semi permanent, stylish and clean cafe on wheels. --> [ 42  12  41 118  33 119  51  47 118 119  96  39 112  54   1  87 111  56
 119  81  19 119  23  87 117  99  47 119]


In [26]:
!head vocab.tsv

ZYXW
shop
just
cheap
go
Indiana
its
We
seems
laid


In [ ]:
# string operations
reviews = tf.constant(lines)
words = tf.string_split(reviews)
densewords = tf.sparse_tensor_to_dense(words, default_value=PADWORD)
numbers = table.lookup(densewords)

# now pad out with zeros and then slice to constant length
padding = tf.constant([[0,0],[0,MAX_DOCUMENT_LENGTH]])
padded = tf.pad(numbers, padding)
sliced = tf.slice(padded, [0,0], [-1, MAX_DOCUMENT_LENGTH])

with tf.Session() as sess:
  tf.tables_initializer().run()
  print "reviews=", reviews.eval(), reviews.shape
  print "words=", words.eval()
  print "dense=", densewords.eval(), densewords.shape
  print "numbers=", numbers.eval(), numbers.shape
  print "padding=", padding.eval(), padding.shape
  print "padded=", padded.eval(), padded.shape
  print "sliced=", sliced.eval(), sliced.shape

<h2> Trenowanie modelu w Cloud ML </h2>

<h2> Serwowanie predykcji </h2>